# The solow model and the solow model with human capital 

> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **model analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `modelproject.py` file includes a function which could be used multiple times in this notebook.

Imports and set magics:

In [6]:
import numpy as np
import sympy as sm
from scipy import optimize
from sympy import Symbol
from sympy.solvers import solve
from IPython.display import display
import matplotlib.pyplot as plt
import ipywidgets as widgets

# output
sm.init_printing(use_unicode=True)

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# output
sm.init_printing(use_unicode=True)

# local modules
import modelproject

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# The basic solow Model 

**Write out the model in equations here.** 

In the basic solow model we have:

$K_t$ as capital\
$L_t$ as labor\


The model also contains\
The GDP\
$Y_t =F(K_t,L_t)$\
$Y_t= B*K_t^\alpha*L_1^{1-\alpha}$\
$r_t=\alpha*B*\left(\frac{K_t}{K_t}\right)$\
$w_t=(1-\alpha) B\left(\frac{K_t}{L_t}\right)^{\alpha}$\
The savings\
$S_t=sY_t,s \in (0,1)$\
$K_{t+1}-K_t=S_t-\delta K_t$\
where we have: $k_t=S_t+(1-\delta)K_t$\
$L_{t+1}=(1+n)L_t$

In the basic solow model we can use this notation for per capita.\

$y_t=\left(\frac{Y_t}{L_t}\right)$\
$k_t=\left(\frac{K_t}{L_t}\right)$
    

## Analytical solution

If your model allows for an analytical solution, you should provide here.

You may use Sympy for this. Then you can characterize the solution as a function of a parameter of the model.

To characterize the solution, first derive a steady state equation as a function of a parameter using Sympy.solve and then turn it into a python function by Sympy.lambdify. See the lecture notes for details. 

In [7]:
#define parameter
n,s,B,k,kstar,ystar,alpha,delta=sm.symbols('n s B k k^* y^* alpha delta')

#Here we will solve the equation for k
Solow_eq = sm.Eq(((s*B*k**alpha-(n + delta)*k)/(1 + n)),0)

#Now it is possible to find the steady-state for k 
k_s=sm.solve(Solow_eq,k)[0]
s_s_k=sm.Eq(kstar,k_s)

# Define y 
y=B*k**alpha

# Find the steady state for y
y_s=y.subs(k,k_s)
s_s_y=sm.Eq(ystar,y_s)

#print equation 
display(s_s_k)
display(s_s_y)

               -1  
              ─────
              α - 1
       ⎛ B⋅s ⎞     
k__* = ⎜─────⎟     
       ⎝δ + n⎠     

                       α
         ⎛        -1  ⎞ 
         ⎜       ─────⎟ 
         ⎜       α - 1⎟ 
         ⎜⎛ B⋅s ⎞     ⎟ 
y__* = B⋅⎜⎜─────⎟     ⎟ 
         ⎝⎝δ + n⎠     ⎠ 

In [8]:
#Turn the equation into function in python
k_s_f= sm.lambdify((alpha, delta, B, n, s), k_s)

In [12]:
#Define the value of the parameteres in the model
n= 0.025
alpha=1/3
delta=0.05
s=0.3
B=1


In [13]:
#The steady state value for k
k_s_v = k_s_f(alpha, delta, B, n, s)

# Print the steady state value for k with the values of the parameters
ss_k_v = sm.Eq(kstar, k_s_v)
sm.pprint(ss_k_v)

k__* = 7.99999999999999


Now it is possible to find the value for y in steady state by using the steady state value for k 

In [14]:
# The steady-state value for y 
y_s_v = B * k_s_v**alpha

# Create the equation for the steady-state y with the computed value
ss_y_e = sm.Eq(ystar, y_s_v)
sm.pprint(ss_y_e)

y__* = 2.0


## Numerical solution

You can always solve a model numerically. 

Define first the set of parameters you need. 

Then choose one of the optimization algorithms that we have gone through in the lectures based on what you think is most fitting for your model.

Are there any problems with convergence? Does the model converge for all starting values? Make a lot of testing to figure these things out. 

In [15]:
def solow_s_s_k(n: float, s: float, B: float, alpha: float, delta: float) -> float:
    """
    Finds the solow steady state for capital by using the bisect optimizer from module Scipy

    Args :
    alpha: Output elasticity wrt capital
    delta: Physical capital depreciation rate
    s    : Savings rate
    n    : Population growth
    B    : Productivity parameter
    """

    # The olow euqation
    solow_eq = lambda k: s * B * k**alpha - (n + delta) * k
    
    # optimize
    return optimize.bisect(solow_eq, 1, 100)

k_star = solow_s_s_k(n, s, B, alpha, delta)
print(f'k* = {k_star:.3f}')

k* = 8.000


In [17]:
# Define function to plot Solow diagram
def plot_solow_diagram(k: int, n: float, s: float, B: int, alpha: float, delta: float, kt_xmax: int, kt_vline: float):
    # Calculate the growth rates of capital per capita and the diagonal line
    k_growth = [s * B * t**alpha for t in range(kt_xmax + 1)]
    diagonal = [(n + delta) * t for t in range(kt_xmax + 1)]

    # Calculate steady state capital per capita
    k_star = ((s*B)/(n+delta))**(1/(1-alpha))

    # Configure plot settings
    plt.rcParams['axes.facecolor'] = 'white'
    plt.figure(figsize=(8, 6))
    plt.plot(k_growth, label=r'$sBk_t^{\alpha}$', color='darkorange')
    plt.plot(diagonal, label=r'$(n+\delta)k_t$', color='black')
    plt.axvline(x=k_star, linestyle='--', color='red', label=r'$k*$')
    plt.axvline(x=kt_vline, linestyle='--', color='green', label=r'$k_t$')
    plt.xlim(0, kt_xmax)
    plt.xlabel('Capital per capita, $k_t$')
    plt.ylabel('')
    plt.legend()
    plt.title('Figure 1: Solow Diagram')
    plt.grid(True)  # add grid
    
    # Add arrows to the plot
    arrow_len = abs(k_star - kt_vline)
    arrow_positions = [0.3, 0.6, 0.8, 0.9, 0.95, 1]
    
    # If kt_vline is to the left of k_star, flip the direction of the arrows
    if kt_vline < k_star:
        arrow_len = -arrow_len
        arrow_positions = [1 - pos for pos in arrow_positions]
    else:
        arrow_positions = [1 - pos for pos in arrow_positions]
    
    arrow_center = min(k_star, kt_vline) + abs(arrow_len) / 2
    
    # Add each arrow to the plot
    for pos in arrow_positions:
        arrow_x = arrow_center + arrow_len * (pos - 0.5)
        plt.annotate(
            '',
            xy=(kt_vline, 0), xycoords='data',
            xytext=(arrow_x, 0), textcoords='data',
            arrowprops=dict(
                arrowstyle="<-", color="black", lw=1, mutation_scale=15
            )
        )

    # print plot
    plt.show()

# Create an interactive widget to display the Solow diagram
widgets.interact(
    plot_solow_diagram,
    k=widgets.fixed(0),
    alpha=widgets.FloatSlider(description=r'α', min=0, max=0.9, step=0.05, value=0.3, layout=widgets.Layout(width='50%')),
    delta=widgets.FloatSlider(description=r'δ', min=0, max=0.1, step=0.01, value=0.01, layout=widgets.Layout(width='50%')),
    s=widgets.FloatSlider(description='s', min=0.01, max=0.8, step=0.05, value=0.4, layout=widgets.Layout(width='50%')),
    n=widgets.FloatSlider(description='n', min=0.01, max=0.1, step=0.005, value=0.01, layout=widgets.Layout(width='50%')),
    B=widgets.fixed(1),
    kt_xmax=widgets.IntSlider(description='k_t axis', min=1, max=100, step=10, value=100, layout=widgets.Layout(width='50%')),
    kt_vline=widgets.FloatSlider(description='k_t', min=0, max=100, step=0.1, value=20, layout=widgets.Layout(width='50%'))
)

interactive(children=(FloatSlider(value=0.01, description='n', layout=Layout(width='50%'), max=0.1, min=0.01, …

<function __main__.plot_solow_diagram(k: int, n: float, s: float, B: int, alpha: float, delta: float, kt_xmax: int, kt_vline: float)>

# Further analysis

Make detailed vizualizations of how your model changes with parameter values. 

Try to make an extension of the model. 

# Conclusion

Add concise conclusion. 